The Agents SDK has a small set of primitives:
- **Agent**: which are LLMs equipeed with instructions and tools 
- **Handoffs**: wich allow agents to delegate to other agents
- **Guardrails**: which enable the inputs to agents to be validated.

In [8]:
from agents import Agent, Runner

agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = await Runner.run(agent, "Write a haiku about Statistics Canada")

print(result.final_output)

Counting every voice,  
Data paints the nation's tale—  
Canada's essence.


# Agents
Agents are the core building blocks in our apps. An agent is LLM configured with instructions and tools. 
The most common properties we configure are:
- `instructions`: also known as a developer message or system prompt.
- `model:` which LLM to use, and `model_settings` to configure tuning praramter
- `tools`: tools that the agent can use to achieve its tasks

In [12]:
from agents import Agent, ModelSettings, function_tool

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="o3-mini",
    tools=[get_weather],
)


result = await Runner.run(
    agent,
    input=f"Tell me about StatCan",
)

print("Final output:", result.final_output)

In [16]:
from agents import Agent, ModelSettings, function_tool, Runner

# A function tool that mimics retrieving basic metadata about a StatCan survey
@function_tool
def get_survey_info(survey_name: str) -> str:
    surveys = {
        "Labour Force Survey": "The Labour Force Survey (LFS) provides estimates of employment and unemployment.",
        "Census": "The Census collects demographic, social, and economic information about everyone living in Canada.",
        "National Household Survey": "The NHS collects detailed information on immigration, education, and housing."
    }
    return surveys.get(survey_name, "Sorry, no information available for that survey.")

# Configure the agent to behave like a bilingual survey metadata assistant
agent = Agent(
    name="Survey Info Agent",
    instructions="You are a bilingual (English and French) assistant. Answer questions about surveys at Statistics Canada.",
    model="o3-mini",
    tools=[get_survey_info],
)

# Run the agent on a prompt related to StatCan
result = await Runner.run(
    agent,
    input="What is the Labour Force Survey?",
)

print("Final output:", result.final_output)

Final output: The Labour Force Survey (LFS) is a monthly survey conducted by Statistics Canada that provides essential estimates of employment, unemployment, and other key labour market indicators. It is the primary source for monitoring the labour market in Canada, enabling policymakers, researchers, and the public to gain insights into employment trends and economic health.

L'enquête sur la population active (EPA) est une enquête mensuelle menée par Statistique Canada qui fournit des estimations essentielles en matière d'emploi, de chômage et d'autres indicateurs clés du marché du travail. Elle constitue la principale source d'information pour suivre l'état du marché du travail au Canada, permettant aux décideurs, aux chercheurs et au public d'obtenir des éclaircissements sur les tendances de l'emploi et la santé économique.


The current example technically uses the function tool (get_survey_info), but the user can’t tell whether the information came from the model itself or from the tool. This obscures the value of tool use, which is critical for understanding how agents augment model capabilities with structured, reliable, or authoritative sources—something extremely relevant at StatsCan.


In [15]:
from agents import Agent, function_tool, Runner

@function_tool
def get_survey_info(survey_name: str) -> str:
    surveys = {
        "Labour Force Survey": "The Labour Force Survey (LFS) provides monthly estimates of employment and unemployment in Canada.",
        "Census": "The Census collects demographic, social, and economic information about the Canadian population.",
    }
    return surveys.get(survey_name, "No official survey found by that name.")

agent = Agent(
    name="Survey Metadata Agent",
    instructions="""
        You are a Statistics Canada assistant that only uses trusted tools to answer questions about surveys.
        Always cite the official description returned by the get_survey_info tool.
        Wrap the tool result in [TOOL OUTPUT] tags to show where the info comes from.
        """,
    model="o3-mini",
    tools=[get_survey_info],
)

result = await Runner.run(
    agent,
    input="Can you give me a trusted description of the Census?",
)

print("Final output:", result.final_output)

Final output: The Census collects demographic, social, and economic information about the Canadian population. [TOOL OUTPUT: "The Census collects demographic, social, and economic information about the Canadian population."]


# Context
In classic dependency injection you build a single container of resources (database handles, config flags, helper mentods) and inject it into every component that needs them. The Agents SDK's generic `context` type formalises exactly that pattern: any dataclass or Pydantic model can act as a DI container. 

**Constructing and passing context**:
Create the object deterministically in our application layer, perhaps by reading user auth, timestamps, or feature flags and pass it to the the runner:

```
ctx = SurveyContext(analyst_id=request.user.id,
                    survey_year=datetime.now().year)
result = await Runner.run(agent, input=user_q, context=ctx)

```

---
Agents are generic on their `context` type. Context is a dependency injection tool: it's an object you create and pass to `Runner.run()`, that is passed to every agent, tool, handoff etc, and serves as a grab bag of dependencies and state for the agent. We can provide any Python object as the context. 

### What is Dependency Injection?
Dependency injection is a pattern for delivering the resources your code needs (configurations, helper methods).

Without DI each tool or function constructs its own dependencies (e.g., database connections, API clients) or pulls them from globals. This scatters setup code and makes testing harder. 

With DI we define a single object (the context) that bundles all dependecies. We then inject that model into every tool, hook, or agent run. All compoents simply reach into the context for what they need. 


In Agents SDK, context is a core **dependency injection primitive**: a typed container of values
and helper methods that your application layer populates before running an agent. We as the developer determinsitically construct this context (drawing on environment variables, user input, database lookups) and pass it to `Runner.run(...)`. Under the hood, SDK wraps it it in a `RunContextWrapper`, which every tool, hook, and handoff recieves, allowing them to access prepopulated data without having to construct or import it themselves. Other agentic frameworks follow a simillar pattern: `LangGraph` offers a `deps_type/deps` injection system.

#### LangGraph

1. Define a Pydantic model for dependencies
```python
from pydantic import BaseModel

class AppDeps(BaseModel):
    db_client: DatabaseClient
    cache: CacheClient
```

2. When building your graph or assistant, specify deps_type
``` python
from langgraph import StateGraph, PlatformAssistant

graph = StateGraph(state_type=YourState, deps_type=AppDeps)
assistant = PlatformAssistant(graph=graph)
```
3. Instantiate and register your dependencies
``` python
deps = AppDeps(db_client=my_db, cache=my_cache)
assistant.register_dependencies(deps)
```
4.	LangGraph injects deps into each node function

```python
@graph.node("fetch_data")
async def fetch_data_node(
    state: YourState,
    deps: AppDeps
) -> YourState:
    record = await deps.db_client.query(...)
    deps.cache.set(..., record)
    return state.update_with(record)
```



### Example: Injecting Context into a StatsCan Conversation

In [30]:
from dataclasses import dataclass
from agents import Agent, function_tool, Runner

@dataclass
class SurveyContext:
    analyst_id: str       # mandatory—no “default” surprises
    survey_year: int

    async def log_lookup(self, survey_name: str):
        # Audit: who looked up what and when
        print(f"[LOG] {self.analyst_id} looked up '{survey_name}'")

@function_tool
async def get_survey_definition(context: SurveyContext, survey_name: str) -> str:
    await context.log_lookup(survey_name)
    catalogue = {
        "Census":  f"The Census {context.survey_year} collects demographic data.",
        "LFS":     f"The Labour Force Survey {context.survey_year} tracks employment trends."
    }
    return catalogue.get(survey_name, "Survey not found.")

agent = Agent[SurveyContext](
    name="StatsCanSurveyAgent",
    instructions="Answer only by calling get_survey_definition.",
    model="o3-mini",
    tools=[get_survey_definition],
)

# — Run 1: Hanan
ctx1 = SurveyContext(analyst_id="Hanan", survey_year=2025)
res1 = await Runner.run(agent, input="What does the Census cover?", context=ctx1)
print(res1.final_output)
# [LOG] Hanan looked up 'Census'
# "The Census 2025 collects demographic data."

# — Run 2: Nick
ctx2 = SurveyContext(analyst_id="Nick", survey_year=2025)
res2 = await Runner.run(agent, input="Give me details on LFS.", context=ctx2)
print(res2.final_output)


Creating trace Agent workflow with id trace_eba8bf05a7fd4533b084ec460fa20aa8
Creating trace Agent workflow with id trace_eba8bf05a7fd4533b084ec460fa20aa8
Setting current trace: trace_eba8bf05a7fd4533b084ec460fa20aa8
Setting current trace: trace_eba8bf05a7fd4533b084ec460fa20aa8
Creating span <agents.tracing.span_data.AgentSpanData object at 0x1092b2f30> with id None
Creating span <agents.tracing.span_data.AgentSpanData object at 0x1092b2f30> with id None
Running agent StatsCanSurveyAgent (turn 1)
Running agent StatsCanSurveyAgent (turn 1)
Creating span <agents.tracing.span_data.ResponseSpanData object at 0x108c5cbd0> with id None
Creating span <agents.tracing.span_data.ResponseSpanData object at 0x108c5cbd0> with id None
Calling LLM o3-mini with input:
[
  {
    "content": "What does the Census cover?",
    "role": "user"
  }
]
Tools:
[
  {
    "name": "get_survey_definition",
    "parameters": {
      "$defs": {
        "SurveyContext": {
          "properties": {
            "analyst_

CancelledError: 

In [29]:
from dataclasses import dataclass
from agents import Agent, function_tool, Runner, enable_verbose_stdout_logging

# 1. Define your context with no hidden defaults
@dataclass
class SurveyContext:
    analyst_id: str
    survey_year: int

    async def log_lookup(self, survey_name: str):
        print(f"[LOG] {self.analyst_id} looked up '{survey_name}'")

# 2. Tool definition remains the same
@function_tool
async def get_survey_definition(context: SurveyContext, survey_name: str) -> str:
    await context.log_lookup(survey_name)
    ctx: SurveyContext = Runner.get_context()
    catalogue = {
        "Census":  f"The Census {context.survey_year} collects demographic data.",
        "LFS":     f"The Labour Force Survey {context.survey_year} tracks employment trends."
    }
    return catalogue.get(survey_name, "Survey not found.")

# 3. Enable verbose logging to inspect LLM ↔ tool interactions
enable_verbose_stdout_logging()

# 4. Use GPT-4.1 (or "gpt-4.1-mini" for a lighter footprint)
agent = Agent[SurveyContext](
    name="StatsCanSurveyAgent",
    instructions="Always call get_survey_definition and wrap output in [SURVEY INFO].",
    model="gpt-4.1",               # updated to the newest model
    tools=[get_survey_definition],
)

# 5. Pass your real analyst IDs—no defaults!
ctx_hanan = SurveyContext(analyst_id="Hanan", survey_year=2025)
result = await Runner.run(
    agent,
    input="What does the Census cover?",
    context=ctx_hanan
)
print(result.final_output)
# [LOG] Hanan looked up 'Census'
# → "[SURVEY INFO] The Census 2025 collects demographic data."

Creating trace Agent workflow with id trace_9a0dbd3684b54a6eb2bfbb9c1735cbcb
Creating trace Agent workflow with id trace_9a0dbd3684b54a6eb2bfbb9c1735cbcb
Setting current trace: trace_9a0dbd3684b54a6eb2bfbb9c1735cbcb
Setting current trace: trace_9a0dbd3684b54a6eb2bfbb9c1735cbcb
Creating span <agents.tracing.span_data.AgentSpanData object at 0x1092b3930> with id None
Creating span <agents.tracing.span_data.AgentSpanData object at 0x1092b3930> with id None
Running agent StatsCanSurveyAgent (turn 1)
Running agent StatsCanSurveyAgent (turn 1)
Creating span <agents.tracing.span_data.ResponseSpanData object at 0x108102d10> with id None
Creating span <agents.tracing.span_data.ResponseSpanData object at 0x108102d10> with id None
Calling LLM gpt-4.1 with input:
[
  {
    "content": "What does the Census cover?",
    "role": "user"
  }
]
Tools:
[
  {
    "name": "get_survey_definition",
    "parameters": {
      "$defs": {
        "SurveyContext": {
          "properties": {
            "analyst_

# Agent Lifecycle Logging

In this section, we define a class called CustomAgentHooks that extends the base class AgentHooks. This class 
is not strictly required for the logic of the agent, but it’s very helpful for debugging and educational purposes. It allows us to see what the agent is doing at each stage of its lifecycle.

## Purpose of Hooks
When working with agent-basded systems, many things happen behind ghetto scenes:
- when an agent starts processing an input
- when it finishes
- when it calls a tool
- when it hands off control to another

These lifecycle stages are invisible unless we explicitly track them. That’s the motivation for creating CustomAgentHooks — it gives us clear, step-by-step feedback on what the agent is doing.}

In [5]:
import asyncio
import random
from typing import Any

from pydantic import BaseModel

from agents import Agent, AgentHooks, RunContextWrapper, Runner, Tool, function_tool

In [8]:
class CustomAgentHooks(AgentHooks):
    def __init__(self, display_name: str):
        self.event_counter = 0
        self.display_name = display_name

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started")

    async def on_end(self, context: RunContextWrapper, agent: Agent, output: Any) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} ended with output {output}"
        )

    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {source.name} handed off to {agent.name}"
        )

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} started tool {tool.name}"
        )

    async def on_tool_end(
        self, context: RunContextWrapper, agent: Agent, tool: Tool, result: str
    ) -> None:
        self.event_counter += 1
        print(
            f"### ({self.display_name}) {self.event_counter}: Agent {agent.name} ended tool {tool.name} with result {result}"
        )

This initializes a hook object. Each instance has:
- event_counter: tracks how many steps have occurred.
- display_name: used to label the logs (e.g., “Start Agent”, “Multiply Agent”).

Each method is asynchronous (async def) because it hooks into an asynchronous system. Here’s what each one does:

In [9]:
@function_tool
def random_number(max: int) -> int:
    """
    Generate a random number up to the provided maximum.
    """
    return random.randint(0, max)

@function_tool
def multiply_by_two(x: int) -> int:
    """Simple multiplication by two."""
    return x * 2

class FinalResult(BaseModel):
    number: int


In [10]:
multiply_agent = Agent(
    name="Multiply Agent",
    instructions="Multiply the number by 2 and then return the final result.",
    tools=[multiply_by_two],
    output_type=FinalResult,
    hooks=CustomAgentHooks(display_name="Multiply Agent"),
)

start_agent = Agent(
    name="Start Agent",
    instructions="Generate a random number. If it's even, stop. If it's odd, hand off to the multiply agent.",
    tools=[random_number],
    output_type=FinalResult,
    handoffs=[multiply_agent],
    hooks=CustomAgentHooks(display_name="Start Agent"),
)

In [11]:
user_input = 100  # You can change this value as needed

result = await Runner.run(
    start_agent,
    input=f"Generate a random number between 0 and {user_input}.",
)

print("Final output:", result.final_output)

### (Start Agent) 1: Agent Start Agent started


/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/contextlib.py:136: RuntimeWarning: coroutine 'main' was never awaited
  def __enter__(self):


### (Start Agent) 2: Agent Start Agent started tool random_number
### (Start Agent) 3: Agent Start Agent ended tool random_number with result 12
### (Start Agent) 4: Agent Start Agent ended with output number=12
Final output: number=12
